In [1]:
import os
import category_encoders as ce
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
#from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LinearRegression, SGDRegressor, Lasso, Ridge, ElasticNet
from sklearn.base import clone
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import StackingRegressor

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
HEROKU_URL = os.getenv('HEROKU_POSTGRESQL_AMBER_URL')

uri = HEROKU_URL 
if uri.startswith("postgres://"):
    uri = uri.replace("postgres://", "postgresql://", 1)

In [3]:
# reading data
def read_data():
    df_raw = df_raw = pd.read_sql('petfinder_with_dates', uri)  
    return df_raw

In [4]:
# dropping irrelevant columns
def feature_drop(df):
    df = df.drop(columns=["id", "name", "organization_id", "published_at", "status_changed_at", "attribute_declawed", "city", "state"])
    return df

In [5]:
#handling missing values

def handling_missing_values(df):
    # Fill NaN values in 'age' column with 'Unknown'
    df['age'].fillna('Unknown', inplace=True)
    return df

In [6]:
def preprocess_data(df):
    # transform "age" column mapping age and size
    age_dict={
    'Baby':'0',
    'Young':'1',
    'Adult':'2',
    'Senior':'3'
    }
    df['age'] = df['age'].map(age_dict).astype(str).astype(int)

    # transform "size" column
    size_dict={
    'Small':'0',
    'Medium':'1',
    'Large': '2',
    'Extra Large': '3'
    }
    df['size'] = df['size'].map(size_dict).astype(str).astype(int)

    # Convert binary columns to binary (0/1) data type
    binary_cols = ["breed_mixed", "breed_unknown", "good_with_children", "good_with_dogs", "good_with_cats", "attribute_spayed_neutered",
                   "attribute_house_trained", "attribute_shots_current", "attribute_special_needs"]
    df[binary_cols] = df[binary_cols].astype(bool).astype(int)

    # Replace 'Male' and 'Female' with 0 and 1, respectively
    df['gender'] = df['gender'].replace({"Male": 0, "Female": 1})

    # Compute the mode of the 'gender' column, ignoring 'Unknown'
    mode = df.loc[df['gender'] != 'Unknown', 'gender'].mode()[0]

    # Replace 'Unknown' values with the mode
    df['gender'] = df['gender'].replace({'Unknown': mode})

    # target encoding on larger categorical features
    target_cols = ["coat", "organization_name", "breed_primary", "breed_secondary", "color_primary", "color_secondary", "color_tertiary"]
    te = ce.TargetEncoder(cols=target_cols)
    df[target_cols] = te.fit_transform(df[target_cols], df["los"])

    return df


In [7]:
# remove outliers
def remove_outliers(df, columns, zscore_threshold=3):
    for col in columns:
        mean = df[col].mean()
        std = df[col].std()
        z_scores = np.abs((df[col] - mean) / std)
        df = df[z_scores <= zscore_threshold]
    return df

In [8]:
def identify_correlated(df, target, threshold):
    # Exclude the target variable from correlation analysis
    features = df.drop(target, axis=1)
    
    corr_matrix = features.corr().abs()
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    reduced_corr_matrix = corr_matrix.mask(mask)
    features_to_drop = [c for c in reduced_corr_matrix.columns if any(reduced_corr_matrix[c] > threshold)]
    return features_to_drop


In [9]:
# split data into training and testing data
def split_data(X, y, test_size = .33, random_state=312):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test

In [10]:
# # Feature selection using RandomForestRegressor
# def feature_selection(X_train, y_train, threshold=0.01):
#     rf = RandomForestRegressor(n_estimators=100, random_state=1)
#     rf.fit(X_train, y_train)

#     # Select features with importance greater than 0.01
#     selector = SelectFromModel(rf, threshold=threshold, prefit=True)
#     X_train_important = selector.transform(X_train)
    
#     return X_train_important

In [11]:
# Perform Randomized Search
def perform_randomized_search(model, param_distributions, X_train, y_train, scoring='r2', cv=5):
    random_search = RandomizedSearchCV(model, param_distributions=param_distributions, n_iter=50, scoring=scoring, cv=cv, n_jobs=-1, random_state=0)
    random_search.fit(X_train, y_train)
    best_model = random_search.best_estimator_
    
    return best_model

In [12]:
# Hyperparameter tuning
def perform_hyperparameter_tuning(pipeline, param_grid, X_train, y_train, cv=3):
    randomized_search = RandomizedSearchCV(pipeline, param_grid, n_iter=10, cv=cv, n_jobs=-1, random_state=1)
    randomized_search.fit(X_train, y_train)
    best_params = randomized_search.best_params_
    best_score = randomized_search.best_score_
    return best_params, best_score

In [13]:
# train and eval models

def train_eval_models(X_train, X_test, y_train, y_test, models):
    results = {}
    for model in models:
        # Create a pipeline to scale the features and initialize the model
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('model', clone(model))
        ])
        
        # Perform cross-validation with additional metrics
        scores_r2 = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='r2')
        scores_mae = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')
        scores_mse = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

        mean_score_r2 = scores_r2.mean()
        mean_score_mae = -scores_mae.mean()
        mean_score_mse = -scores_mse.mean()

        # Model name and store results with each model
        name = model.__class__.__name__
        results[name] = (mean_score_r2, mean_score_mae, mean_score_mse)
        print('{} done. Mean R-squared (CV): {:.2f}, Mean MAE (CV): {:.2f}, Mean MSE (CV): {:.2f}'.format(
            name, mean_score_r2, mean_score_mae, mean_score_mse))
        
    # Train the best model on the entire training set and evaluate on the test set

    best_model = max(results, key=results.get)
    best_pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('model', clone(models[models.index(best_model)]))
    ])
    best_pipeline.fit(X_train, y_train)
    y_test_pred = best_pipeline.predict(X_test)

    print('\nBest model: {}'.format(best_model))
    print('R-squared (test set): {:.2f}'.format(r2_score(y_test, y_test_pred)))
    print('Mean squared error (test set): {:.2f}'.format(mean_squared_error(y_test, y_test_pred)))
    print('Mean absolute error (test set): {:.2f}'.format(mean_absolute_error(y_test, y_test_pred)))


In [14]:
# def train_eval_models_important_features(X_train_important, X_test_important, y_train, y_test, models):
#     # Retrain models with important features
#     results_important = {}

#     for model in models:
#         # Initialize the models
#         regr = model
#         regr.fit(X_train_important, y_train)
        
#         # Predictions from models
#         y_test_pred = regr.predict(X_test_important)
        
#         # Model name and stored results with each model
#         name = model.__class__.__name__
        
#         results_important[name] = r2_score(y_test, y_test_pred)
#         print('{} done. R-squared (important features): {:.2f}'.format(name, results_important[name]))

#     # Find the best model
#     best_model = max(results_important, key=results_important.get)
#     best_r2 = results_important[best_model]

#     # Evaluate the best model
#     best_regr = models[[m.__class__.__name__ for m in models].index(best_model)]
#     y_test_pred = best_regr.predict(X_test_important)

#     mse = mean_squared_error(y_test, y_test_pred)
#     mae = mean_absolute_error(y_test, y_test_pred)

#     print('\nBest model: {}'.format(best_model))
#     print('R-squared (test set, important features): {:.2f}'.format(best_r2))
#     print('Mean squared error (test set, important features): {:.2f}'.format(mse))
#     print('Mean absolute error (test set, important features): {:.2f}'.format(mae))


In [15]:
# Read data from the database
df_raw = read_data()

In [16]:
# Feature drop
df_raw = feature_drop(df_raw)

In [17]:
# Handle missing values
df_raw = handling_missing_values(df_raw)

In [18]:
# Preprocess data
df_preprocessed = preprocess_data(df_raw)

In [19]:
# Remove outliers
outlier_columns = ['organization_name', 'los', 'breed_primary', 'breed_secondary']
df_no_outliers = remove_outliers(df_preprocessed, outlier_columns)

In [20]:
# Identify correlated features
correlation_threshold = 0.2
to_drop = identify_correlated(df_no_outliers, target="los", threshold=correlation_threshold)
df = df_no_outliers.drop(to_drop, axis=1)

In [24]:
# Split data into training and testing sets
X = df.drop('los', axis = 1)
y = df['los']
X_train, X_test, y_train, y_test = split_data(X, y)

In [25]:
# # Feature selection
# feature_selection_threshold = 0.01
# X_train_important = feature_selection(X_train,y_train, feature_selection)
# X_test_important = feature_selection(X_test, y_test, feature_selection_threshold)

In [26]:
# Define models
models = [SGDRegressor(random_state=0), 
          GradientBoostingRegressor(random_state=0), 
          LinearRegression(),
          Lasso(random_state=0),
          Ridge(random_state=0),
          ElasticNet(random_state=0),
          DecisionTreeRegressor(random_state=0),
          RandomForestRegressor(n_estimators=100, random_state=0),
          XGBRegressor(),
          LGBMRegressor()]

In [27]:
# perform randomizedsearchCV for XGBRegressor
xgb_param_distributions = {
    'n_estimators': [50, 100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5, 6, 7],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.5, 0.7, 0.9],
    'colsample_bytree': [0.5, 0.7, 0.9],
    'gamma': [0, 0.1, 0.2]
}
# best_xgb = perform_randomized_search(XGBRegressor(random_state=0), xgb_param_distributions,
#                                      X_train_important, y_train)

best_xgb = perform_randomized_search(XGBRegressor(random_state=0), xgb_param_distributions,
                                     X_train, y_train)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [28]:
# Hyperparameter tuning for RandomForestRegressor
params_rf = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [3, 4, 5, 6, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt']
}

# best_rf_params, _ = perform_hyperparameter_tuning(RandomForestRegressor(random_state=0), params_rf,
#                                                   X_train_important, y_train)
best_rf_params, _ = perform_hyperparameter_tuning(RandomForestRegressor(random_state=0), params_rf,
                                                   X_train, y_train)

best_rf = RandomForestRegressor(**best_rf_params, random_state=0)

/opt/anaconda3/envs/myenv_petfinder/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/opt/anaconda3/envs/myenv_petfinder/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/opt/anaconda3/envs/myenv_petfinder/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove t

In [29]:
# Hyperparameter tuning for GradientBoostingRegressor

gbr_param_grid = {
    'model__n_estimators': [100, 200, 300],
    'model__learning_rate': [0.01, 0.1, 0.2],
    'model__max_depth': [3, 4, 5],
    'model__min_samples_split': [2, 3, 4],
    'model__min_samples_leaf': [1, 2, 3]
}

gbr_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', GradientBoostingRegressor())
])

# best_gbr_params, best_gbr_score = perform_hyperparameter_tuning(gbr_pipeline, gbr_param_grid, 
#                                                                 X_train_important, y_train)
best_gbr_params, best_gbr_score = perform_hyperparameter_tuning(gbr_pipeline, gbr_param_grid, 
                                                                X_train, y_train)
print('GradientBoostingRegressor best parameters: ', best_gbr_params)
print('GradientBoostingRegressor best score: {:.2f}'.format(best_gbr_score))

GradientBoostingRegressor best parameters:  {'model__n_estimators': 300, 'model__min_samples_split': 4, 'model__min_samples_leaf': 3, 'model__max_depth': 5, 'model__learning_rate': 0.2}
GradientBoostingRegressor best score: 0.61


In [30]:
# Hyperparameter tuning for LGBMRegressor
lgbm_param_grid = {
    'model__n_estimators': [100, 200, 300],
    'model__learning_rate': [0.01, 0.1, 0.2],
    'model__max_depth': [3, 4, 5],
    'model__num_leaves': [31, 45, 60],
    'model__min_child_samples': [20, 30, 40]
}

lgbm_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LGBMRegressor())
])

# best_lgbm_params, best_lgbm_score = perform_hyperparameter_tuning(lgbm_pipeline, lgbm_param_grid, 
#                                                                   X_train_important, y_train)
best_lgbm_params, best_lgbm_score = perform_hyperparameter_tuning(lgbm_pipeline, lgbm_param_grid, 
                                                                  X_train, y_train)

print('LGBMRegressor best parameters: ', best_lgbm_params)
print('LGBMRegressor best score: {:.2f}'.format(best_lgbm_score))

LGBMRegressor best parameters:  {'model__num_leaves': 60, 'model__n_estimators': 300, 'model__min_child_samples': 40, 'model__max_depth': 5, 'model__learning_rate': 0.2}
LGBMRegressor best score: 0.48


In [31]:
# Perform Stacking
estimators = [
    ('ridge', Ridge()),
    ('lasso', Lasso()),
    ('elastic_net', ElasticNet()),
    ('gbm', GradientBoostingRegressor()),
    ('xgb', XGBRegressor()),
    ('lgbm', LGBMRegressor())
]
stacking_regressor = StackingRegressor(estimators=estimators, final_estimator=Ridge())
models.append(stacking_regressor)

In [32]:
# # Train and evaluate models
# train_eval_models(X_train_important, X_test_important, y_train, y_test, models)
train_eval_models(X_train, X_test, y_train, y_test, models)

# # Retrain models with important features and evaluate again
# train_eval_models_important_features(X_train_important, X_test_important, y_train, y_test, models)

SGDRegressor done. Mean R-squared (CV): 0.13, Mean MAE (CV): 64.49, Mean MSE (CV): 13710.29
GradientBoostingRegressor done. Mean R-squared (CV): 0.36, Mean MAE (CV): 53.35, Mean MSE (CV): 10076.34
LinearRegression done. Mean R-squared (CV): 0.13, Mean MAE (CV): 64.05, Mean MSE (CV): 13696.09
Lasso done. Mean R-squared (CV): 0.13, Mean MAE (CV): 63.65, Mean MSE (CV): 13705.27
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been de

TypeError: Cannot clone object. You should provide an instance of scikit-learn estimator instead of a class.